In [162]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

In [163]:
df = pd.DataFrame()
onlyfiles = [f for f in listdir('./') if isfile(join('./', f)) and '.csv' in f and f != 'dataframe.csv']

for file in onlyfiles:
    df = pd.concat([df,pd.read_csv('./' + file)],axis=0,ignore_index=True)

In [171]:
print('shape_pre:',df.shape)

shape_pre: (297, 15)


In [165]:
# drop duplicates
idx = df[['local date','lat','lon']].drop_duplicates().index.tolist()
df = df.iloc[idx].reset_index(drop=True)

# drop NA location
df = df.dropna(subset=['location'])

# drop local time
df = df.drop(['local time'], axis=1)

# drop location
df = df.drop(['location'], axis=1)

# drop dew
df = df.drop(['dew'], axis=1)

In [166]:
# location for date
location_for_date={}
for i in range(len(df)):
    if df.iloc[i]['local date'] in location_for_date:
        location_for_date[df.iloc[i]['local date']].append((df.iloc[i]['lat'],df.iloc[i]['lon']))
    else:
        location_for_date[df.iloc[i]['local date']] = [(df.iloc[i]['lat'],df.iloc[i]['lon'])]
        
dates = df['local date'].drop_duplicates().tolist()

min_number_location = set(location_for_date['2020-03-30'])
max_number_location = set(location_for_date['2020-03-30'])
for date in dates:
    min_number_location = min_number_location.intersection(set(location_for_date[date]))
    max_number_location = max_number_location.union(set(location_for_date[date]))
    
# inserisco posti mancanti per data
for date in location_for_date:
    for lat_lon in max_number_location:
        if lat_lon not in location_for_date[date]:
            df = df.append(pd.DataFrame([{'local date': date, 'lat': lat_lon[0], 'lon': lat_lon[1]}]))
            
df = df.sort_values(by='local date').reset_index(drop=True)

In [172]:
print('shape_post',df.shape)

shape_post (297, 15)


In [170]:
print(df.head())

   local date        lat        lon aqi   co     h   no2    o3       p  pm10  \
0  2020-03-30  41.907183  12.446688  65  0.1  65.4   7.8  19.9  1009.4  56.0   
1  2020-03-30  41.893267  12.475385  58  NaN  65.4   7.8  20.4  1009.4  58.0   
2  2020-03-30  41.993969  12.724505  61  NaN  62.0   6.0  19.5  1009.9  46.0   
3  2020-03-30  41.964406  12.668440  65  NaN  61.2   6.4  19.5  1010.9  45.0   
4  2020-03-30  41.932832  12.501247  70  0.1  65.4  10.6  18.7  1009.4  55.0   

   pm25  so2     t    w   wg  
0  65.0  0.5  17.2  1.2  4.8  
1  25.0  NaN  17.2  1.2  4.8  
2  61.0  1.0  15.5  NaN  NaN  
3  65.0  1.0  16.8  1.5  4.2  
4  70.0  0.5  17.2  1.2  4.8  


In [176]:
print('NA values:\n')
print(df.isnull().sum(axis = 0))

NA values:

local date      0
lat             0
lon             0
aqi            12
co            140
h              12
no2            12
o3             12
p              12
pm10           12
pm25           12
so2            44
t              12
w              48
wg             48
dtype: int64


In [179]:
df.to_csv('dataframe.csv', index=False)